## Importing Packages

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


## Scoring Model Best Cities to work remote

### Numbeo

To build our scoring model, we are going to use Numbeo. This website is the largest contributed database about cities and is a tremendous tool that we are going to use extensively to gather all the data that are interesting to define a good city for remote workers.
On the technical side, we are going to use the package BeautifulSoup to scrape the data from the website. The data that we are going to scrape are in two different pages. Therefore we are going to create two dataframes and merge them ultimately

In [1]:
# First page: Cost of Living data

import pandas as pd
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://www.numbeo.com/cost-of-living/rankings_current.jsp').text
soup = BeautifulSoup(website_url,'html.parser')
content_table = soup.find('table',{'class':'stripe row-border order-column compact'})

cities = []
for td in content_table.find_all('td')[1::8]:
    cities.append(td.text)
cost_of_living_index = []
for td in content_table.find_all('td')[2::8]:
    cost_of_living_index.append(td.text)
rent_index = []
for td in content_table.find_all('td')[3::8]:
    rent_index.append(td.text)
cost_of_living_plus_rent_index = []
for td in content_table.find_all('td')[4::8]:
    cost_of_living_plus_rent_index.append(td.text)
groceries_index = []
for td in content_table.find_all('td')[5::8]:
    groceries_index.append(td.text)
restaurant_price_index = []
for td in content_table.find_all('td')[6::8]:
    restaurant_price_index.append(td.text)
local_purchasing_power_index = []
for td in content_table.find_all('td')[7::8]:
    local_purchasing_power_index.append(td.text)

data = {'City':cities, 'Cost of Living Index':cost_of_living_index, 'Rent Index': rent_index, 
        'Cost of Living Plus Rent Index': cost_of_living_plus_rent_index, 'Groceries Index': groceries_index,
        'Restaurant Price Index': restaurant_price_index, 'Local Purchasing Power Index': local_purchasing_power_index}

df1 = pd.DataFrame(data)
df1.head()

,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,"Basel, Switzerland",128.33,46.43,89.50,131.93,112.94,108.76
1,"Zurich, Switzerland",125.57,62.96,95.88,126.29,109.21,124.61
2,"Lausanne, Switzerland",122.55,52.62,89.40,129.41,104.10,104.97
3,"Geneva, Switzerland",118.79,66.92,94.20,117.24,107.27,111.93
4,"Bern, Switzerland",112.97,41.40,79.04,105.30,103.46,129.30


In [2]:
#We only take only the most interesting feature for us: Cost of Living Plus Rent Index, Groceries Index, Restaurant Price Index
df1 = df1[['City','Cost of Living Plus Rent Index', 'Groceries Index', 'Restaurant Price Index']]
df1.head()

,City,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index
0,"Basel, Switzerland",89.50,131.93,112.94
1,"Zurich, Switzerland",95.88,126.29,109.21
2,"Lausanne, Switzerland",89.40,129.41,104.10
3,"Geneva, Switzerland",94.20,117.24,107.27
4,"Bern, Switzerland",79.04,105.30,103.46


In [3]:
df1[df1.City=='Bali, Indonesia']

,City,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index
288,"Bali, Indonesia",26.94,41.25,22.21


In [4]:
# Second Page: Quality of Life data

website_url = requests.get('https://www.numbeo.com/quality-of-life/rankings_current.jsp').text
soup = BeautifulSoup(website_url,'html.parser')
content_table = soup.find('table',{'class':'stripe row-border order-column compact'})

content_table
cities = []
for td in content_table.find_all('td')[1::11]:
    cities.append(td.text)
safety_index = []
for td in content_table.find_all('td')[4::11]:
    safety_index.append(td.text)
healthcare_index = []
for td in content_table.find_all('td')[5::11]:
    healthcare_index.append(td.text)
pollution_index = []
for td in content_table.find_all('td')[9::11]:
    pollution_index.append(td.text)
climate_index = []
for td in content_table.find_all('td')[10::11]:
    climate_index.append(td.text)

data = {'City':cities, 'Safety Index':safety_index, 'Healthcare Index': healthcare_index, 
        'Pollution Index': pollution_index, 'Climate Index': climate_index}
df2 = pd.DataFrame(data)
df2.head()

,City,Safety Index,Healthcare Index,Pollution Index,Climate Index
0,"Canberra, Australia",79.33,82.17,14.07,82.72
1,"Adelaide, Australia",71.63,81.22,18.31,94.96
2,"Raleigh, NC, United States",66.17,75.62,21.87,83.88
3,"Wellington, New Zealand",70.78,74.90,13.66,97.68
4,"Columbus, OH, United States",57.87,74.28,25.19,71.29


In [5]:
# Bali, which is an important destination for remote workers is missing in this page. That is why we are going to add it
# in the dataframe by taking the data from the closest city, Jakarta. We know that this might not be very acurate because of the 
# difference between the two places but it seems the closest one. A mean or a median of the whole dataset would have been less 
# accurate.

dfBali=pd.DataFrame()

dfBali = df2[df2.City=='Jakarta, Indonesia']
dfBali = dfBali.replace(to_replace ="Jakarta, Indonesia",value ="Bali, Indonesia") 
df2 = df2.append(dfBali, ignore_index=True)
df2.head()


,City,Safety Index,Healthcare Index,Pollution Index,Climate Index
0,"Canberra, Australia",79.33,82.17,14.07,82.72
1,"Adelaide, Australia",71.63,81.22,18.31,94.96
2,"Raleigh, NC, United States",66.17,75.62,21.87,83.88
3,"Wellington, New Zealand",70.78,74.90,13.66,97.68
4,"Columbus, OH, United States",57.87,74.28,25.19,71.29


In [6]:
# Finally we join the two dataframes
df_ = df1.merge(df2, on = 'City')
df_.head()

,City,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index
0,"Zurich, Switzerland",95.88,126.29,109.21,83.11,73.69,17.77,81.48
1,"Geneva, Switzerland",94.20,117.24,107.27,71.41,68.89,29.26,82.61
2,"New York, NY, United States",100.00,100.00,100.00,54.32,63.33,56.34,79.66
3,"San Francisco, CA, United States",102.32,86.54,83.05,45.07,66.02,46.33,97.26
4,"Anchorage, AK, United States",64.49,87.60,67.81,37.21,60.62,16.62,41.61


In [7]:
# So far we have a lot of data. But we would like to include data like internet speed and % of remote workers because we think 
# that these is two factors are essential to make a good city for remote workers.
# The dataset that we encountered are made with distinct columns for cities and countries. That is why, we are going to 
# manipulate the column of our dataframe to best fit the other dataset that we are going to be using.

df_city_country = df_.City.str.split(',', expand = True)
df_city_country.columns = ['City','Country1','Country2']
df_city_country.replace('(^\s+|\s+$)', '', regex=True, inplace=True)
df_city_country['Country1'][df_city_country.Country2 == 'United States'] = 'United States'
df_city_country = df_city_country[['City','Country1']]
df_city_country.columns = ['City','Country']
df_city_country.head()

,City,Country
0,Zurich,Switzerland
1,Geneva,Switzerland
2,New York,United States
3,San Francisco,United States
4,Anchorage,United States


In [8]:
# We put these new columns in our dataframe with a join
df_ = df_.iloc[:,1:]
df_ = df_city_country.join(df_)
df_.head()

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index
0,Zurich,Switzerland,95.88,126.29,109.21,83.11,73.69,17.77,81.48
1,Geneva,Switzerland,94.20,117.24,107.27,71.41,68.89,29.26,82.61
2,New York,United States,100.00,100.00,100.00,54.32,63.33,56.34,79.66
3,San Francisco,United States,102.32,86.54,83.05,45.07,66.02,46.33,97.26
4,Anchorage,United States,64.49,87.60,67.81,37.21,60.62,16.62,41.61


### Cable Internet Speed 

 A good internet connection is essential for a remote worker. Therefore we are going to use the data from a Research designed and compiled by Cable.co.uk, and gathered by M-Lab, an open source project with contributors from civil society organisations, educational institutions, and private sector companies.

In [35]:
df_internet_speed = pd.read_excel(r'C:\Users\WeWork-4\Downloads\worldwide-broadband-speed-league-2019-data.xlsx', index_col=0)
df_internet_speed= df_internet_speed[:207].reset_index()
df_internet_speed= df_internet_speed[['Country','Position (2019) – Out of 207', 'Mean download speed (2019)', 'Percentage increase/decrease (2018-2019)']]
df_internet_speed.columns = ['Country','Ranking 2019', 'Speed', '% change (2018-2019)']

#We change some country names to match with the names of the dataframe df_test
df_internet_speed['Country'] = df_internet_speed['Country'].replace({'Czechia':'Czech Republic', 'Bosnia and Herzegovina': 'Bosnia And Herzegovina','Republic of Korea':'South Korea', 'Republic of Moldova':'Moldova'})
df_internet_speed.columns

Index(['Country', 'Ranking 2019', 'Speed', '% change (2018-2019)'], dtype='object')

In [36]:
df_with_internet = df_.merge(df_internet_speed, on = 'Country')
df_raw = df_with_internet
df_with_internet.head()

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Ranking 2019,Speed,% change (2018-2019)
0,Zurich,Switzerland,95.88,126.29,109.21,83.11,73.69,17.77,81.48,9.0,38.853625,0.298618
1,Geneva,Switzerland,94.20,117.24,107.27,71.41,68.89,29.26,82.61,9.0,38.853625,0.298618
2,New York,United States,100.00,100.00,100.00,54.32,63.33,56.34,79.66,15.0,32.887566,0.271809
3,San Francisco,United States,102.32,86.54,83.05,45.07,66.02,46.33,97.26,15.0,32.887566,0.271809
4,Anchorage,United States,64.49,87.60,67.81,37.21,60.62,16.62,41.61,15.0,32.887566,0.271809


### Pourcentage of Nomad in the City

Social life is important also for the remote workers. In fact, it is nice to meet some like minded people onsite to share experiences and have fun. The website NomadList gathered an impressive amount of data from digital nomads. The one we are going to be using is the percentage of nomads per population.  
Scraping the pages of NomadList is quite difficult, therefore we gathered the data manually in an Excel Sheet. It was long and fastidious but much needed for our model.

In [37]:
dfNomadPlaces=pd.read_excel(r'C:\Users\WeWork-4\Downloads\NomadPlaces.xlsx')
dfNomadPlaces.head()

,City,Cntry,Pct Nomads
0,Bali,Indonesia,27.0
1,Ko Pha Ngan,Thailand,4.0
2,Bocas del Toro,Panama,2.0
3,Chiang Mai,Thailand,0.3
4,Whistler,Canada,0.2


In [47]:
df_all_data = df_with_internet.merge(dfNomadPlaces, on='City', how='left').iloc[:,[0,1,2,3,4,5,6,7,8,10,13]]
df_all_data['Pct Nomads'].fillna(0, inplace=True)
df_all_data = df_all_data.rename(columns={'Speed': 'Internet Index'})
df_all_data['Internet Index'] = round(df_all_data['Internet Index'].astype('float'),2)
df_all_data

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Internet Index,Pct Nomads
0,Zurich,Switzerland,95.88,126.29,109.21,83.11,73.69,17.77,81.48,38.85,0.1
1,Geneva,Switzerland,94.20,117.24,107.27,71.41,68.89,29.26,82.61,38.85,0.1
2,New York,United States,100.00,100.00,100.00,54.32,63.33,56.34,79.66,32.89,0.1
3,San Francisco,United States,102.32,86.54,83.05,45.07,66.02,46.33,97.26,32.89,0.1
4,Anchorage,United States,64.49,87.60,67.81,37.21,60.62,16.62,41.61,32.89,0.0
...,...,...,...,...,...,...,...,...,...,...,...
235,Bogota,Colombia,19.29,21.85,19.04,38.61,64.36,69.61,97.12,3.48,0.0
236,Medellin,Colombia,17.53,23.26,17.52,48.41,76.59,63.02,99.76,3.48,0.1
237,Lahore,Pakistan,14.11,19.97,15.08,60.71,64.61,79.19,67.56,1.44,0.0
238,Islamabad,Pakistan,14.73,20.37,15.55,70.77,64.53,43.64,76.91,1.44,0.0


### Data Normalizing



The columns of our dataframe have completely different scales. Therefore, we are going to rescale the dataframe to change the values of its columns to a common scale, without distorting differences in the ranges of values. We will be using the normalization method that rescales the values into a range of [0.1].
For that purpose we are going to use the data normalization method of scikit learn

In [39]:
from sklearn import preprocessing

df_normalized = df_all_data

x = df_normalized.values[:,2:] #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_normalized.loc[:,2:] = x_scaled
df_normalized.head()

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Internet Index,Pct Nomads
0,Zurich,Switzerland,0.928856,1.000000,1.000000,0.923212,0.731142,0.060070,0.784588,0.448057,0.003704
1,Geneva,Switzerland,0.910296,0.915357,0.980614,0.764762,0.630004,0.198387,0.797782,0.448057,0.003704
2,New York,United States,0.974370,0.754115,0.907964,0.533315,0.512853,0.524377,0.763339,0.376808,0.003704
3,San Francisco,United States,1.000000,0.628227,0.738583,0.408044,0.569532,0.403876,0.968827,0.376808,0.003704
4,Anchorage,United States,0.582081,0.638141,0.586290,0.301598,0.455752,0.046226,0.319089,0.376808,0.000000


In [40]:
#The values of the columns 'Cost of Living Plus Rent Index','Groceries Index', 'Restaurant Price Index', 'Safety Index'
#'Pollution Index' need to be inversed

df_normalized['Cost of Living Plus Rent Index'] = 1- df_normalized['Cost of Living Plus Rent Index']
df_normalized['Groceries Index']= 1 - df_normalized['Groceries Index']
df_normalized['Restaurant Price Index']= 1- df_normalized['Restaurant Price Index']
df_normalized['Safety Index']= df_normalized['Safety Index']
df_normalized['Pollution Index']= 1 - df_normalized['Pollution Index']
df_normalized.head()

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Internet Index,Pct Nomads
0,Zurich,Switzerland,7.114450e-02,1.110223e-16,0.000000,0.923212,0.731142,0.939930,0.784588,0.448057,0.003704
1,Geneva,Switzerland,8.970393e-02,8.464272e-02,0.019386,0.764762,0.630004,0.801613,0.797782,0.448057,0.003704
2,New York,United States,2.562970e-02,2.458848e-01,0.092036,0.533315,0.512853,0.475623,0.763339,0.376808,0.003704
3,San Francisco,United States,2.220446e-16,3.717733e-01,0.261417,0.408044,0.569532,0.596124,0.968827,0.376808,0.003704
4,Anchorage,United States,4.179187e-01,3.618593e-01,0.413710,0.301598,0.455752,0.953774,0.319089,0.376808,0.000000


### Total Score

Some factores may be more important that others. We consider that the climate, the restaurant prices and the percentage of digital nomads in the city are the most important ones. That's why we are going to give them a weight twice more important that the others. 
Lastly, we sum the factors to get our final score. 

In [41]:
#Most important factor count double
df_normalized['Climate Index'] = df_normalized['Climate Index']*2
df_normalized['Restaurant Price Index'] = df_normalized['Restaurant Price Index']*2
df_normalized['Pct Nomads'] = df_normalized['Pct Nomads']*2
df_normalized['Total Score'] = df_normalized.sum(axis=1)
df_total = df_normalized.sort_values(by='Total Score',ascending=False)
df_total.reset_index(drop=True).head(20)

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Internet Index,Pct Nomads,Total Score
0,Taipei,Taiwan,0.657976,0.458848,1.660038,0.964112,1.000000,0.557361,1.636894,1.000000,0.007407,7.942636
1,Bursa,Turkey,0.927640,0.926394,1.803737,0.789680,0.826169,0.663416,1.847285,0.046623,0.000000,7.830946
2,Izmir,Turkey,0.932722,0.943509,1.828920,0.773023,0.716603,0.425545,1.928780,0.046623,0.000000,7.595725
3,Bali,Indonesia,0.832744,0.795361,1.738783,0.427546,0.372314,0.137956,1.155400,0.063120,2.000000,7.523224
4,Valencia,Spain,0.724702,0.802469,1.285300,0.806609,0.899284,0.622487,1.857793,0.414704,0.000000,7.413348
5,Medellin,Colombia,0.936699,0.963618,1.832517,0.453277,0.792246,0.395209,1.996030,0.025224,0.007407,7.402228
6,Curitiba,Brazil,0.930623,0.974373,1.789148,0.295368,0.566372,0.767907,1.992294,0.041482,0.000000,7.357567
7,Lisbon,Portugal,0.685484,0.828283,1.396622,0.774919,0.684787,0.734320,1.968710,0.255589,0.007407,7.336121
8,Porto,Portugal,0.729452,0.815657,1.469172,0.655065,0.750737,0.714337,1.922475,0.255589,0.000000,7.312484
9,Timisoara,Romania,0.884666,0.914141,1.669831,0.831798,0.683312,0.474660,1.582487,0.244232,0.000000,7.285128


There we are! We got finally our results. In the top 20, we can find some well-known cities for remote workers like  Bali, Medellin, Lisbon, Prague or Chiang Mai. There are some surprise also, with cities like Bursa and Izmir in Turkey, Timisoara and Cluk in Romania, Islamad in Pakistan or Coimbatore in India. In can be explained by the facts that cost of living is quite cheap and safety and healthcare are quite good and have improved substantially in those regions. 

## Clustering 

Now that we have all the data available, we can also think of find the similarities between the cities. To do so, we are going to cluster them thanks to k-means model. It is especially useful when it comes to quickly discover insights from unlabeled data.

### K-Means Model

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
import numpy as np


# for that analysis, we are going to take into account only factors from the Numbeo pages 
# (without the % of digital nomads in the population)
df_cluster = df_all_data
X = df_cluster.values[:,2:-2]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 2.89516576,  3.75164235,  2.97094072, ..., -1.56639068,
         0.24556048,  1.3504594 ],
       [ 2.80525302,  3.31176892,  2.87937043, ..., -1.02310715,
         0.31128108,  1.3504594 ],
       [ 3.11566606,  2.47382221,  2.53621784, ...,  0.25732088,
         0.13970959,  0.90910013],
       ...,
       [-1.48112288, -1.41601978, -1.47210653, ...,  1.33774113,
        -0.56402429, -1.41988462],
       [-1.44794079, -1.39657786, -1.44992197, ..., -0.34317528,
        -0.02022993, -1.41988462],
       [-1.50413626, -1.44518266, -1.3951686 , ...,  1.84508946,
        -0.34534335, -1.41988462]])

In [93]:
num_clusters = 4

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_

print(k_means_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 0 0 0 0 2 0 2 0 0 2 2 2 2 0 0 2 0 0 0 0 2 0 2 2 2 2 2 2 2 2 0 0 2 2 2 2 0
 0 2 2 2 2 2 2 2 0 2 2 0 2 0 2 2 2 2 0 2 2 2 0 2 2 0 0 2 2 2 2 2 2 2 2 0 2
 0 2 2 2 2 3 2 2 1 2 3 3 1 2 2 2 3 3 1 3 2 3 3 2 2 3 3 3 2 2 3 3 3 2 2 3 1
 1 2 2 1 2 2 1 1 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 1 1 1 3 3 3 3 3 1 1 1 1 1 1 1 1 3 3 3 3 3 3 1 1 1 1 1 3 3 1 1 1 1 3 3
 3 3 1 3 3 3 3 3 3 3 3 3 3 1 1 3 3 3]


In [94]:
# We add a label column with the cluster number of each city
df_cluster["Labels"] = k_means_labels
df_cluster.head()

,City,Country,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Safety Index,Healthcare Index,Pollution Index,Climate Index,Internet Index,Pct Nomads,Labels
0,Zurich,Switzerland,95.88,126.29,109.21,83.11,73.69,17.77,81.48,38.85,0.1,0
1,Geneva,Switzerland,94.20,117.24,107.27,71.41,68.89,29.26,82.61,38.85,0.1,0
2,New York,United States,100.00,100.00,100.00,54.32,63.33,56.34,79.66,32.89,0.1,0
3,San Francisco,United States,102.32,86.54,83.05,45.07,66.02,46.33,97.26,32.89,0.1,0
4,Anchorage,United States,64.49,87.60,67.81,37.21,60.62,16.62,41.61,32.89,0.0,0


### Coordinates Cities

In [95]:
df_coordinates = pd.read_csv(r'C:\Users\WeWork-4\Downloads\Copy of worldcities.csv')
df_coordinates.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Abohar,Abohar,30.1204,74.2900,India,IN,IND,Punjab,NaN,130603.0,1356846795
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


In [96]:
df_cluster_coordinates = df_cluster.merge(df_coordinates, left_on = ['City','Country'], right_on = ['city_ascii','country'], how = 'left')
df_cluster_coordinates = df_cluster_coordinates.iloc[:,[0,1,11,14,15,21]]
df_cluster_coordinates= df_cluster_coordinates.dropna()
df_cluster_coordinates

,City,Country,Labels,lat,lng,population
0,Zurich,Switzerland,0,47.3800,8.5500,1108000.0
1,Geneva,Switzerland,0,46.2100,6.1400,1240000.0
2,New York,United States,0,40.6943,-73.9249,19354922.0
3,San Francisco,United States,0,37.7562,-122.4430,3603761.0
4,Anchorage,United States,0,61.1508,-149.1091,253421.0
...,...,...,...,...,...,...
235,Bogota,Colombia,1,4.5964,-74.0833,7772000.0
236,Medellin,Colombia,1,6.2750,-75.5750,3297000.0
237,Lahore,Pakistan,3,31.5600,74.3500,6577000.0
238,Islamabad,Pakistan,3,33.7000,73.1666,780000.0


In [97]:
# We can see that the 5 most populated cities in the world belong to different clusters according to our model
df_cluster_coordinates[df_cluster_coordinates.population >15000000]

,City,Country,Labels,lat,lng,population
2,New York,United States,0,40.6943,-73.9249,19354922.0
49,Tokyo,Japan,2,35.6850,139.7514,35676000.0
195,Sao Paulo,Brazil,1,-23.5587,-46.6250,18845000.0
217,Mexico City,Mexico,1,19.4424,-99.1310,19028000.0
221,Mumbai,India,3,19.0170,72.8570,18978000.0
222,Delhi,India,3,28.6700,77.2300,15926000.0


### Map

In [109]:
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(zoom_start=3)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_cluster_coordinates['lat'], df_cluster_coordinates['lng'], df_cluster_coordinates['Labels']):
    label = folium.Popup('Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
#Make a data frame with dots to show on the map
df_best_cities = df_cluster_coordinates[df_cluster_coordinates.City.isin(['Taipei', 'Bursa', 'Izmir', 'Valencia', 'Medellin', 'Bali'])]

# I can add marker one by one on the map
for i in range(0,len(df_best_cities)):
    folium.Marker([df_best_cities.iloc[i]['lat'], df_best_cities.iloc[i]['lng']], popup=df_best_cities.iloc[i]['City']).add_to(map_clusters)
       
map_clusters

This map provide interesting caracteristics.The data points from the same cluster are located next to each other. The green points are in majority in Asia. The purple points in Latin America and Africa. The red and blue points are located in the so called 'developped countries'.

What is interesting too, is that the cities in the top 6 of our scoring model don't belong to only one clusters but they are spread out into different clusters: 
- Taipei and Valencia in the blue cluster
- Bali in the green cluster
- Burza, Izmir and Medellin in the purple cluster

The only cluster not represented here is the red cluster where the cost of living is relatively higher than the quality of life.

# Annex

## Foursquare

Now that we know the best spots to work remotely, we may want to know where we are going to sip a nice cocktail or coffee and get work done. Co-working spaces are nice places but the majority are costly and don't offer the freedom to work from one place to another. On the other hand, coffee shops have nowadays have an internet connexion, large range of drinks, chilled music and space to work with a computer. That is why we may want to know if the city has various coffee places before booking our flight.

Let's explore one of the most trendy city for digital nomads, Lisbon, thanks to the Foursquare API.

In [130]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

CLIENT_ID = 'GI2JXJT4SZPDO2O34A0KMJJ0J2WX32AALRL23VY5XVLOGTGN' # your Foursquare ID
CLIENT_SECRET = 'NI1CGZYFALYEH2PFLHFKWINZONMOCWXNI4VHSYKIFCEOW0BY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

address = 'Lisbon, Portugal'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

search_query = 'Cafe'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea60fb978a484001b01939f'},
 'response': {'venues': [{'id': '588a5d0b5490d32cb99c50c4',
    'name': 'Delirium Café Lisboa',
    'location': {'address': 'Calçada Nova de São Francisco, 2A',
     'lat': 38.71040780026144,
     'lng': -9.139451702571572,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.71040780026144,
       'lng': -9.139451702571572}],
     'distance': 386,
     'cc': 'PT',
     'city': 'Lisboa',
     'state': 'Lisboa',
     'country': 'Portugal',
     'formattedAddress': ['Calçada Nova de São Francisco, 2A',
      'Lisboa',
      'Portugal']},
    'categories': [{'id': '56aa371ce4b08b9a8d57356c',
      'name': 'Beer Bar',
      'pluralName': 'Beer Bars',
      'shortName': 'Beer Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587941393',
    'hasPerk': False},
   {'id': '4dd417f422716ea3ce55f1cb',
    'name': 

In [134]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']

dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,crossStreet,id
0,Delirium Café Lisboa,Beer Bar,"Calçada Nova de São Francisco, 2A",38.710408,-9.139452,"[{'label': 'display', 'lat': 38.71040780026144...",386,PT,Lisboa,Lisboa,Portugal,"[Calçada Nova de São Francisco, 2A, Lisboa, Po...",NaN,NaN,NaN,588a5d0b5490d32cb99c50c4
1,Café Vitória,Coffee Shop,Rua da Vitória,38.711070,-9.137474,"[{'label': 'display', 'lat': 38.71106967476913...",377,PT,Lisboa,Lisboa,Portugal,"[Rua da Vitória, Lisboa, Portugal]",NaN,NaN,NaN,4dd417f422716ea3ce55f1cb
2,Há Café no Alfarrabista,Café,"Rua da Madalena, 80D",38.710438,-9.135214,"[{'label': 'display', 'lat': 38.71043829929498...",322,PT,Lisboa,Lisboa,Portugal,"[Rua da Madalena, 80D, 1100-322 Lis Lisboa, Po...",1100-322 Lis,NaN,NaN,51b0b346498e3fd204c8e038
3,Penta Café,Coffee Shop,"Rua do Ouro, 115",38.710224,-9.138376,"[{'label': 'display', 'lat': 38.71022379234900...",315,PT,Lisboa,Lisboa,Portugal,"[Rua do Ouro, 115, 1100-048 Lisboa, Portugal]",1100-048,NaN,NaN,4ee75d42e4b04d23e3e14099
4,Pois Café,Café,"R. São João Praça, 93-95",38.709488,-9.131935,"[{'label': 'display', 'lat': 38.709488, 'lng':...",448,PT,Lisboa,Lisboa,Portugal,"[R. São João Praça, 93-95, 1100-521 Lisboa, Po...",1100-521,NaN,NaN,4bd329279854d13afd05fd4d


In [132]:
dataframe_filtered.name

0                   Delirium Café Lisboa
1                           Café Vitória
2                Há Café no Alfarrabista
3                             Penta Café
4                              Pois Café
5                             Praça Café
6                Café Martinho da Arcada
7                    Café Central Lisboa
8        Café Teatro Santiago Alquimista
9                           Café Ribeiro
10                        Café no Chiado
11                          Café Guarany
12                                Cafe 3
13    Hamburgueria Gourmet - Café do Rio
14                           Café Lisboa
15                     Cruzes Credo Café
16                              Café 22A
17                             Café Tofa
18                          Finezzo Café
19                           Café Nicola
20                            Cafe Peter
21                      O Moinho Do Café
22                              cafe pit
23                           Houria Café
24              

In [133]:
 venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel
# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

As we can see, the city offers a large range of coffee shops located next to each other so we might work one day in Delerium Cafe and in Cafe Martinho the other day